Importing and installing all that can be necessary

In [2]:
import numpy as np 
import pandas as pd 
import types
from botocore.client import Config
import ibm_boto3
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [6]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.21.0-py_0 conda-forge


geographiclib-1.50   | 34 KB     | ##################################### | 100% 
geopy-1.21.0         | 58 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

Preparing the Foursquare setting

In [7]:
#Foursquare
CLIENT_ID = 'WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL' # your Foursquare ID
CLIENT_SECRET = '0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO' # your Foursquare Secret
VERSION = '20200422' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL
CLIENT_SECRET:0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO


We are setting in Toronto

In [8]:
address = 'Toronto, TO'
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Preparing the map of Toronto

In [9]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

In [13]:
#check
#[map_toronto

Let's explore Toronto Neighborhood using Foursquare platform

In [14]:
LIMIT = 100 
radius = 500 
#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WS3IH3T0F5EXCVUTHBJFHJRJMJJHUV2P1WPOJPCPC4CQDOZL&client_secret=0ASEX1X35VF41WVO0NH1XAXPTARBPG5LPFL4DUY35HCK2VOO&v=20200422&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [15]:
results = requests.get(url).json()

Let's divide what we found in categories

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
        
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

#filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

#filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Indigo,Bookstore,43.653515,-79.380696


Now we define what are the most common categories for each Neighborhood

First, we prepare the Data of Toronto getting them from Wikipedia

In [17]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.drop(df[df['Borough']=='Not assigned'].index,inplace=True)
df.reindex

<bound method DataFrame.reindex of     Postal code           Borough  \
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
5           M6A        North York   
6           M7A  Downtown Toronto   
8           M9A         Etobicoke   
9           M1B       Scarborough   
11          M3B        North York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M6B        North York   
17          M9B         Etobicoke   
18          M1C       Scarborough   
20          M3C        North York   
21          M4C         East York   
22          M5C  Downtown Toronto   
23          M6C              York   
26          M9C         Etobicoke   
27          M1E       Scarborough   
30          M4E      East Toronto   
31          M5E  Downtown Toronto   
32          M6E              York   
36          M1G       Scarborough   
39          M4G         East York   
40          M5G  Downtown Toronto   
41 

In [18]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3b5c2bdc58c94defb31bb5be8c5e2af7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='huIpeVpPXm4bVzCZtfEBR68OfhcR5zhM0nI0r1THmmWy',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_3b5c2bdc58c94defb31bb5be8c5e2af7.get_object(Bucket='ibmcourseradatasciencemodule9caps-donotdelete-pr-pfzgdxkvbxscse',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_data_0 = pd.read_csv(body)
df_data_0.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
df_final = pd.merge(left=df, right=df_data_0, left_on='Postal code', right_on='Postal Code')
del df_final['Postal Code']
df_final.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


Then we are ready to get nearby venues

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
Toronto_venues = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )
                                  
print(Toronto_venues.shape)
Toronto_venues.head()

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [21]:
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
#Toronto_grouped
Toronto_grouped.shape

(94, 267)

In [114]:
nearby_venues['categories']

0                      Neighborhood
1                             Plaza
2                    Breakfast Spot
3                  Sushi Restaurant
4                         Bookstore
5                    Cosmetics Shop
6                   Bubble Tea Shop
7                     Shopping Mall
8                        Restaurant
9                       Coffee Shop
10                     Concert Hall
11                   Cosmetics Shop
12                       Art Museum
13             Fast Food Restaurant
14                   Clothing Store
15                 Ramen Restaurant
16    Vegetarian / Vegan Restaurant
17                          Theater
18                       Restaurant
19                   Clothing Store
20                            Hotel
21              Japanese Restaurant
22              American Restaurant
23                            Plaza
24           Furniture / Home Store
25                          Theater
26               Seafood Restaurant
27               Seafood Res

We have a total of 70 categories

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 70

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Chinese Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Colombian Restaurant,College Rec Center,Greek Restaurant
1,Alderwood / Long Branch,Pizza Place,Sandwich Place,Skating Rink,Gym,Coffee Shop,Pub,Pharmacy,Women's Store,Dessert Shop,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Bank,Grocery Store,Fried Chicken Joint,Sandwich Place,Bridal Shop,Diner,Restaurant,Ice Cream Shop,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Colombian Restaurant,College Gym,College Rec Center
4,Bedford Park / Lawrence Manor East,Coffee Shop,Italian Restaurant,Restaurant,Sandwich Place,Sushi Restaurant,Greek Restaurant,Indian Restaurant,Juice Bar,Liquor Store,...,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop


Now, Using machine learning K-means algorithm, let's subdivide the totality of data in clusters

In [25]:
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df_final

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Doner Restaurant,Dim Sum Restaurant,...,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,College Rec Center,College Gym,Greek Restaurant,Auto Workshop,Beer Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Stadium,College Gym,Greek Restaurant
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Bakery,Pub,Park,...,Gastropub,Gas Station,Garden Center,Garden,Gaming Cafe,Accessories Store,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Shoe Store,Miscellaneous Shop,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,College Stadium
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1.0,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,...,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field


In [84]:
#Toronto_merged.dropna(inplace=True)
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

Now we can create a Map and superimpose the markers for each cluster


In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Now we can examine our cluster

In [90]:
#Examine clusters 1
cluster1=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster1.shape
cluster1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
63,York,0,Bus Line,Pizza Place,Breakfast Spot,Convenience Store,Women's Store,Doner Restaurant,Diner,Discount Store,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Stadium,College Cafeteria,College Gym
64,York,0,Convenience Store,Women's Store,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,College Stadium,College Gym,Wings Joint,Auto Workshop,Beer Store


In [101]:
#Examine clusters 2
cluster2=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster2.shape

(86, 72)

In [42]:
#Examine clusters 3
cluster3=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster3.shape

(7, 72)

In [43]:
#Examine clusters 4
cluster4=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster4.shape

(1, 72)

In [44]:
#Examine clusters 5
cluster5=Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
cluster5.shape

(3, 72)

We choose the more significant one, the cluster 2

In [102]:
cluster2.shape

(86, 72)

In [103]:
cluster2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
1,North York,1,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Stadium,College Gym,Greek Restaurant
2,Downtown Toronto,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,...,Gastropub,Gas Station,Garden Center,Garden,Gaming Cafe,Accessories Store,Furniture / Home Store,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
3,North York,1,Clothing Store,Furniture / Home Store,Shoe Store,Miscellaneous Shop,Sporting Goods Shop,Coffee Shop,Boutique,Vietnamese Restaurant,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,College Stadium
4,Downtown Toronto,1,Coffee Shop,Diner,Sushi Restaurant,Yoga Studio,Distribution Center,Café,Beer Bar,Spa,...,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field
6,Scarborough,1,Fast Food Restaurant,Dessert Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,College Stadium,College Gym,Wings Joint,Auto Workshop,Beer Store
7,North York,1,Coffee Shop,Asian Restaurant,Japanese Restaurant,Gym,Beer Store,Restaurant,Concert Hall,Café,...,Food Truck,Food Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant
8,East York,1,Pizza Place,Athletics & Sports,Gym / Fitness Center,Pet Store,Pharmacy,Gastropub,Fast Food Restaurant,Bank,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market,College Stadium,Colombian Restaurant
9,Downtown Toronto,1,Clothing Store,Coffee Shop,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Restaurant,...,Electronics Store,Bakery,Sporting Goods Shop,Ethiopian Restaurant,Comfort Food Restaurant,Convenience Store,Cuban Restaurant,Coworking Space,Concert Hall,Creperie
10,North York,1,Park,Metro Station,Pizza Place,Pub,Japanese Restaurant,Distribution Center,Dessert Shop,Dim Sum Restaurant,...,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant,College Rec Center,Women's Store,College Cafeteria
12,Scarborough,1,Golf Course,Bar,Women's Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,College Stadium,College Gym,Wings Joint,Bike Rental / Bike Share


Finally we found where Greek Restaurant a

In [104]:
cluster2_Ok = cluster2['70th Most Common Venue'].str.find('Greek Restaurant')
cluster2_Ok.sum()


-83

In [105]:
DF_OK = cluster2.loc[cluster2['70th Most Common Venue'] =='Greek Restaurant']

In [106]:
DF_OK

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
1,North York,1,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Stadium,College Gym,Greek Restaurant
49,North York,1,Park,Basketball Court,Bakery,Construction & Landscaping,Diner,Discount Store,Distribution Center,Dog Run,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market,College Gym,Greek Restaurant
78,Scarborough,1,Latin American Restaurant,Lounge,Breakfast Spot,Chinese Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Colombian Restaurant,College Rec Center,Greek Restaurant


In [112]:
Toronto_merged.loc[Toronto_merged['70th Most Common Venue'] =='Greek Restaurant']

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,...,61th Most Common Venue,62th Most Common Venue,63th Most Common Venue,64th Most Common Venue,65th Most Common Venue,66th Most Common Venue,67th Most Common Venue,68th Most Common Venue,69th Most Common Venue,70th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Coffee Shop,Hockey Arena,French Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Stadium,College Gym,Greek Restaurant
49,M6L,North York,North Park / Maple Leaf Park / Upwood Park,43.713756,-79.490074,1,Park,Basketball Court,Bakery,Construction & Landscaping,...,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market,College Gym,Greek Restaurant
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Swim School,Dog Run,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,College Rec Center,Women's Store,Greek Restaurant
78,M1S,Scarborough,Agincourt,43.794200,-79.262029,1,Latin American Restaurant,Lounge,Breakfast Spot,Chinese Restaurant,...,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Fast Food Restaurant,Colombian Restaurant,College Rec Center,Greek Restaurant


# The best are Borough North York Victoria Village or North Park , Maple Leaf Park , Upwood Park 